In [1]:
from keybert import KeyBERT
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import mysql.connector
from sqlalchemy import create_engine
from tqdm import tqdm 

In [2]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Establish a connection using the loaded configuration
cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM combined_reviews"

# Use pandas to load sql query into a DataFrame
dataset = pd.read_sql(query, con=cnx)

# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_21136\4013114179.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(query, con=cnx)


In [3]:
score_sentiment = []
for score in dataset.score:
  if score > 3:
    score_sentiment.append(1) # Positive
  # elif score == 3:
  #   score_sentiment.append(0) # Neutral
  else:
    score_sentiment.append(-1) # Negative

In [4]:
dataset['score_sentiment'] = score_sentiment

In [5]:
dataset['score_sentiment'].value_counts()

-1    4462
 1    1176
Name: score_sentiment, dtype: int64

In [6]:
dataset.head()

,replyContent,score,content,thumbsUpCount,date,score_sentiment
0,Hi there! Thank you for your 5-star rating rev...,5,I like how the app is really light and fast co...,0,NaT,1
1,"Hi Jerry, this feature isn't ready at the mome...",1,Cannot add card to google pay. Also cannot add...,0,NaT,-1
2,Thanks for your feedback on mobile wallets and...,4,Generally good. However please add either NFC ...,0,NaT,1
3,Hi Eric. We're sorry to hear this. We would li...,1,"With continuous failed to log in,contacted cso...",0,NaT,-1
4,Hi Hafidz Melek! Thank you for your 5-star rat...,5,I don't get the negative reviews here. I insta...,0,NaT,1


In [7]:
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Construct the connection string using the loaded configuration
user = config['user']
password = config['password']
host = config['host']
dbname = config['database']
port = config['port']
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}'

# Create an engine
engine = create_engine(conn_string)

dataset.to_sql(name='predetermined_sentiment', con=engine, if_exists='replace', index=False)
#dataset.to_csv("sentiment.csv")

5638

In [8]:
def sentiment_accuracy(model):
  sentiment_pipeline = pipeline("sentiment-analysis", model=model)
  count = 0

  for row in range(len(dataset.content)):
    pred_sentiment = sentiment_pipeline(dataset.content[row])[0]['label']
    pred_sentiment = define(pred_sentiment)
    if pred_sentiment == dataset.score_sentiment[row]:
      count += 1

  acc = count/len(dataset.content)
  return acc

In [9]:
# baseline model
baseline_model = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'

def define(pred):
  if pred == 'POSITIVE': return 1
  # elif pred == 'NEGATIVE': return -1
  else: return -1

sentiment_accuracy(baseline_model)

KeyboardInterrupt: 

In [ ]:
# fin model
fin_model = 'yiyanghkust/finbert-tone'

def define(pred):
  if pred == 'Positive': return 1
  # elif pred == 'Negative': return -1
  else: return -1

sentiment_accuracy(fin_model)

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

0.8441499085923218

## **Finetuning** model to fit our bank reviews better
To fine tune our pre trained models with our data: https://huggingface.co/blog/sentiment-analysis-python
**take note that the labels are 0 and 1 not -1 and 1**


In [10]:
with open('config.json') as config_file:
    config = json.load(config_file)['database']


cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM datasettrain"

# Use pandas to load sql query into a DataFrame
datasettrain = pd.read_sql(query, con=cnx)


# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_21136\3012561626.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasettrain = pd.read_sql(query, con=cnx)


In [11]:
with open('config.json') as config_file:
    config = json.load(config_file)['database']


cnx = mysql.connector.connect(**config)

# Define your query
query = f"SELECT * FROM datasettest"

# Use pandas to load sql query into a DataFrame
datasettest = pd.read_sql(query, con=cnx)


# Don't forget to close the connection when done
cnx.close()

C:\Users\AngJi\AppData\Local\Temp\ipykernel_21136\565115240.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasettest = pd.read_sql(query, con=cnx)


In [12]:
score_sentiment_train = []
for score in datasettrain.score:
  if score > 3:
    score_sentiment_train.append(1) # Positive
  else:
    score_sentiment_train.append(0) # Negative

score_sentiment_test = []
for score in datasettest.score:
  if score > 3:
    score_sentiment_test.append(1) # Positive
  else:
    score_sentiment_test.append(0) # Negative

In [13]:
datasettrain['score_sentiment'] = score_sentiment_train
datasettest['score_sentiment']=score_sentiment_test

In [14]:
datasettrain.head()

,date,score,content,thumbsUpCount,replyContent,score_sentiment
0,2020-03-01 04:14:38,5,Face ID and OTP issues : The OTP to create a n...,0,,1
1,2021-12-23 20:47:02,5,Easy to use & simple to understand : I find th...,0,,1
2,2022-12-12 02:43:20,5,The best digital banking app in Singapore! : I...,0,,1
3,2020-11-09 01:44:36,4,Accessing digital token when logged in. : Well...,0,,1
4,2024-02-13 11:05:23,4,Payment Controls Monthly Spend Tracking : Unde...,0,,1


In [15]:
# baseline model train
baseline_model_train = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'


In [17]:
finetune=datasettrain.drop(columns=['replyContent','thumbsUpCount','score','date'])
finetune=finetune.rename(columns={
    'content':'text',
    'score_sentiment':'label'
})

finetune.head()


,text,label
0,Face ID and OTP issues : The OTP to create a n...,1
1,Easy to use & simple to understand : I find th...,1
2,The best digital banking app in Singapore! : I...,1
3,Accessing digital token when logged in. : Well...,1
4,Payment Controls Monthly Spend Tracking : Unde...,1


In [18]:
len(finetune)

5303

In [19]:
# Load database configuration from JSON file
with open('config.json') as config_file:
    config = json.load(config_file)['database']

# Construct the connection string using the loaded configuration
user = config['user']
password = config['password']
host = config['host']
dbname = config['database']
port = config['port']
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}'

# Create an engine
engine = create_engine(conn_string)

finetune.to_sql(name='finetune', con=engine, if_exists='replace', index=False)

5303

In [20]:
#For fine tuning
!pip install datasets
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Step 1: Load your finetune file into a Pandas DataFrame
df = finetune
# Columns to remove
columns_to_remove = ['Unnamed: 0.1', 'Unnamed: 0', '__index_level_0__']

# Only drop columns which are present in the DataFrame
df = df.drop(columns=[col for col in columns_to_remove if col in df.columns])
# Step 2: Convert the DataFrame to a Hugging Face `Dataset`
dataset_huggingface = Dataset.from_pandas(df)

# Assuming 'df' is your loaded DataFrame and 'dataset' is the Dataset you created

# Split your DataFrame into training and testing sets first
train_df, test_df = train_test_split(df, test_size=0.1)  # For example, 10% as test

# Convert the train and test DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.remove_columns('__index_level_0__')
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.remove_columns('__index_level_0__')

# Convert to DatasetDict (if needed for convenience)
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})
# Shuffle and select small subsets for fine-tuning (as examples)
small_train_dataset = dataset_dict['train'].shuffle(seed=42).select(range(1008))
small_test_dataset = dataset_dict['test'].shuffle(seed=42).select(range(112))

# Example output
print(small_train_dataset[0])
print(small_test_dataset[0])




{'text': 'Can’t login anymore : After the newest feature update of mobile app, I can no longer log in via Face ID Nor via ID n code.', 'label': 0}
{'text': 'After login after updated : Can not login after updated', 'label': 0}


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(baseline_model_train)


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1008 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(baseline_model_train, num_labels=2)

In [ ]:
# Define the evaluation metrics
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 820.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:

#!kill -9 -1
#!pip install accelerate -U
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-bank_reviews-otherbank"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()

In [ ]:
# Run inferences with your new model using Pipeline
from transformers import pipeline

sentiment_model = pipeline(model="ajiayi/finetuning-sentiment-model-bank_reviews-otherbank")

sentiment_model(["I love this move", "This movie sucks!"])

In [ ]:
def sentiment_accuracy_test(model):
  sentiment_pipeline = pipeline("sentiment-analysis", model=model)
  count = 0

  for row in range(len(datasettest.content)):
    pred_sentiment = sentiment_pipeline(datasettest.content[row])[0]['label']
    pred_sentiment = define(pred_sentiment)
    if pred_sentiment == datasettest.score_sentiment[row]:
      count += 1

  acc = count/len(datasettest.content)
  return acc

In [ ]:
#finetuned version of baseline model

# baseline model train
baseline_model_finetune = 'ajiayi/finetuning-sentiment-model-bank_reviews-otherbank'

def define(pred):
  if pred == 'POSITIVE': return 1
  elif pred == 'NEGATIVE': return 0


sentiment_accuracy_test(baseline_model_finetune)

config.json:   0%|          | 0.00/823 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

0.9818731117824774

In [ ]:
import pandas as pd
from transformers import pipeline

def sentiment_analysis_negative_to_df(model_identifier):
    sentiment_pipeline = pipeline("sentiment-analysis", model=model_identifier)
    negative_indices = []

    for idx, row in datasettest.iterrows():
        pred_sentiment = sentiment_pipeline(row['content'])[0]['label']
        if pred_sentiment == 'NEGATIVE':
            negative_indices.append(idx)

    # Using the indices of negative rows to create a new DataFrame
    negative_df = datasettest.loc[negative_indices]

    return negative_df

# Replace 'model_identifier' with your model's identifier.
negative_df = sentiment_analysis_negative_to_df(baseline_model_finetune)
negative_df.head()

#this dataframe contains all negatively sentiment reviews. accuracy should be around 97% based on finetuned model.
#use this dataframe to generate responses to by the gpt. (just an idea)

,replyContent,score,content,thumbsUpCount,score_sentiment
0,"We're sorry to hear that you feel this way, Li...",1,Not sure if i am already registered for the sa...,0,0
1,Hi Kh Lim. We're sorry to hear this. Please co...,1,developer is nutshell brain i think proof of a...,0,0
2,Hi Shaun. We're sorry to hear this and we woul...,1,i cant eve open the app now and purchase thing...,0,0
5,We apologize for the inconvenience you had to ...,3,It always ask for finger identification even t...,0,0
6,"Hi Sean, we understand your concern and we wou...",1,What I've installed on my phone is none of you...,0,0
